# Annotate merged single cells with metadata from platemap file

## Import libraries

In [1]:
import pathlib
import sys

import lancedb
import pandas as pd
from pycytominer import annotate
from pycytominer.cyto_utils import output

/home/lippincm/miniforge3/envs/cellprofiler_timelapse_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Set paths and variables

In [2]:
# load in platemap file as a pandas dataframe
platemap_path = pathlib.Path("../../data/").resolve()

# directory where parquet files are located
data_dir = pathlib.Path("../data/converted_data")

# directory where the annotated parquet files are saved to
output_dir = pathlib.Path("../data/annotated_data")
output_dir.mkdir(exist_ok=True)

In [3]:
# dictionary with each run for the cell type
dict_of_inputs = {
    "run_20231017ChromaLive_6hr_4ch_MaxIP": {
        "source_path": pathlib.Path(
            f"{data_dir}/20231017ChromaLive_6hr_4ch_MaxIP.parquet"
        ).resolve(strict=True),
        "platemap_path": pathlib.Path(f"{platemap_path}/platemap_6hr_4ch.csv").resolve(
            strict=True
        ),
    },
}

## Annotate merged single cells

In [4]:
for data_run, info in dict_of_inputs.items():
    # load in converted parquet file as df to use in annotate function
    single_cell_df = pd.read_parquet(info["source_path"])
    single_cell_df.rename(
        columns={
            "Image_Metadata_FOV": "Metadata_FOV",
            "Image_Metadata_Time": "Metadata_Time",
        },
        inplace=True,
    )
    platemap_df = pd.read_csv(info["platemap_path"])
    output_file = str(pathlib.Path(f"{output_dir}/{data_run}_sc.parquet"))
    print(f"Adding annotations to merged single cells for {data_run}!")

    # add metadata from platemap file to extracted single cell features
    annotated_df = annotate(
        profiles=single_cell_df,
        platemap=platemap_df,
        join_on=["Metadata_well", "Image_Metadata_Well"],
    )

    # move metadata well and single cell count to the front of the df (for easy visualization in python)
    well_column = annotated_df.pop("Metadata_Well")
    singlecell_column = annotated_df.pop("Metadata_number_of_singlecells")
    # insert the column as the second index column in the dataframe
    annotated_df.insert(1, "Metadata_Well", well_column)
    annotated_df.insert(2, "Metadata_number_of_singlecells", singlecell_column)

    # rename metadata columns to match the expected column names
    columns_to_rename = {
        "Nuclei_Location_Center_Y": "Metadata_Nuclei_Location_Center_Y",
        "Nuclei_Location_Center_X": "Metadata_Nuclei_Location_Center_X",
    }
    # Image_FileName cols
    for col in annotated_df.columns:
        if "Image_FileName" in col:
            columns_to_rename[col] = f"Metadata_{col}"
        elif "Image_PathName" in col:
            columns_to_rename[col] = f"Metadata_{col}"
        elif "TrackObjects" in col:
            columns_to_rename[col] = f"Metadata_{col}"
    # rename metadata columns
    annotated_df.rename(columns=columns_to_rename, inplace=True)

    # save annotated df as parquet file
    output(
        df=annotated_df,
        output_filename=output_file,
        output_type="parquet",
    )
    print(f"Annotations have been added to {data_run} and saved!")
    # check last annotated df to see if it has been annotated correctly
    print(annotated_df.shape)
    annotated_df.head()

Adding annotations to merged single cells for run_20231017ChromaLive_6hr_4ch_MaxIP!
Annotations have been added to run_20231017ChromaLive_6hr_4ch_MaxIP and saved!
(19382, 2318)


## Add the object tacking from SAM2

In [5]:
# set and connect to the db
# create the database object
uri = pathlib.Path("../../data/objects_db").resolve()
db = lancedb.connect(uri)

In [6]:
# get the db schema and tables
db.table_names()
# load table
table = db["1.masked_images"]
location_metadata_df = table.to_pandas()
print(location_metadata_df.shape)
location_metadata_df.head()

(14882, 8)


,image_set_name,frame,object_id,x,y,mask_path,mask_file_name,mask_file_path
0,E-11_F0002,0,1,120.634918,15.555555,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...,0.png,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...
1,E-11_F0002,0,2,293.525635,23.012821,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...,0.png,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...
2,E-11_F0002,0,3,1115.543457,20.380434,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...,0.png,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...
3,E-11_F0002,0,4,46.547619,38.988094,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...,0.png,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...
4,E-11_F0002,0,5,1820.597778,37.445652,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...,0.png,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...


In [7]:
# change frame to Metadata_Time
location_metadata_df.rename(columns={"frame": "Metadata_Time"}, inplace=True)
# add 1 to Metadata_Time to match the timepoints in the single cell data
location_metadata_df["Metadata_Time"] = location_metadata_df["Metadata_Time"] + 1
# change formatting to leading 4 zeros
location_metadata_df["Metadata_Time"] = location_metadata_df["Metadata_Time"].apply(
    lambda x: f"{x:04}"
)
print(location_metadata_df.shape)
location_metadata_df.head()

(14882, 8)


,image_set_name,Metadata_Time,object_id,x,y,mask_path,mask_file_name,mask_file_path
0,E-11_F0002,0001,1,120.634918,15.555555,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...,0.png,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...
1,E-11_F0002,0001,2,293.525635,23.012821,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...,0.png,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...
2,E-11_F0002,0001,3,1115.543457,20.380434,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...,0.png,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...
3,E-11_F0002,0001,4,46.547619,38.988094,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...,0.png,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...
4,E-11_F0002,0001,5,1820.597778,37.445652,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...,0.png,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...


In [8]:
print(annotated_df.shape)
annotated_df["Metadata_image_set_name"] = (
    annotated_df["Metadata_Well"].astype(str)
    + "_"
    + "F"
    + annotated_df["Metadata_FOV"].astype(str)
)
image_set_names = annotated_df.pop("Metadata_image_set_name")
# move to front
annotated_df.insert(0, "Metadata_image_set_name", image_set_names)
time = annotated_df.pop("Metadata_Time")
annotated_df.insert(1, "Metadata_Time", time)
x_coord = annotated_df.pop("Metadata_Nuclei_Location_Center_X")
Y_coord = annotated_df.pop("Metadata_Nuclei_Location_Center_Y")
annotated_df.insert(2, "Metadata_Nuclei_Location_Center_X", x_coord)
annotated_df.insert(3, "Metadata_Nuclei_Location_Center_Y", Y_coord)
annotated_df.head()

(19382, 2318)


/tmp/ipykernel_348045/4163974095.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  annotated_df.insert(2, 'Metadata_Nuclei_Location_Center_X', x_coord)
/tmp/ipykernel_348045/4163974095.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  annotated_df.insert(3, 'Metadata_Nuclei_Location_Center_Y', Y_coord)


,Metadata_image_set_name,Metadata_Time,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_plate,Metadata_Well,Metadata_number_of_singlecells,Metadata_compound,Metadata_dose,Metadata_control,...,Nuclei_Texture_Variance_CL_488_2_3_02_256,Nuclei_Texture_Variance_CL_488_2_3_03_256,Nuclei_Texture_Variance_CL_561_3_00_256,Nuclei_Texture_Variance_CL_561_3_01_256,Nuclei_Texture_Variance_CL_561_3_02_256,Nuclei_Texture_Variance_CL_561_3_03_256,Nuclei_Texture_Variance_DNA_3_00_256,Nuclei_Texture_Variance_DNA_3_01_256,Nuclei_Texture_Variance_DNA_3_02_256,Nuclei_Texture_Variance_DNA_3_03_256
0,C-02_F0001,0003,1292.318182,41.272727,1,C-02,151,Staurosporine,0.0,negative,...,0.000000,0.000000,0.184153,0.179238,0.123578,0.097597,0.000000,0.000000,0.000000,0.000000
1,C-02_F0001,0003,1115.739130,60.956522,1,C-02,151,Staurosporine,0.0,negative,...,0.000000,0.000000,0.405343,0.336418,0.345861,0.410430,0.000614,0.000618,0.000599,0.000636
2,C-02_F0001,0003,454.666667,111.666667,1,C-02,151,Staurosporine,0.0,negative,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,C-02_F0001,0003,542.365854,99.682927,1,C-02,151,Staurosporine,0.0,negative,...,0.001831,0.001074,0.904483,0.912830,0.906310,0.891520,0.000000,0.000000,0.000000,0.000000
4,C-02_F0001,0003,701.512195,76.878049,1,C-02,151,Staurosporine,0.0,negative,...,0.288753,0.279669,0.286101,0.285829,0.325779,0.288216,0.591353,0.599560,0.623875,0.597146


In [9]:
# drop NaN values from annotated_df
print(annotated_df.shape)
annotated_df = annotated_df.dropna(
    subset=["Metadata_Nuclei_Location_Center_X", "Metadata_Nuclei_Location_Center_Y"]
)
print(annotated_df.shape)
print(location_metadata_df.shape)

(19382, 2319)
(14575, 2319)
(14882, 8)


In [10]:
# match the x and y coordinates to the image set name in the location metadata df
annotated_df["Metadata_Nuclei_Location_Center_X"] = annotated_df[
    "Metadata_Nuclei_Location_Center_X"
].astype(int)
annotated_df["Metadata_Nuclei_Location_Center_Y"] = annotated_df[
    "Metadata_Nuclei_Location_Center_Y"
].astype(int)
location_metadata_df["x"] = location_metadata_df["x"].astype(int)
location_metadata_df["y"] = location_metadata_df["y"].astype(int)

merged_df = annotated_df.merge(
    location_metadata_df,
    how="left",
    left_on=[
        "Metadata_Nuclei_Location_Center_X",
        "Metadata_Nuclei_Location_Center_Y",
        "Metadata_Time",
        "Metadata_image_set_name",
    ],
    right_on=["x", "y", "Metadata_Time", "image_set_name"],
)
print(merged_df.shape)
# sort by image_set_name and Metadata_Time
merged_df = merged_df.sort_values(by=["Metadata_image_set_name", "Metadata_Time"])
# drop right columns
merged_df = merged_df.drop(
    columns=[
        "image_set_name",
        "object_id",
        "x",
        "y",
        "mask_path",
        "mask_file_name",
        "mask_file_path",
    ]
)
merged_df.head()

(14575, 2326)


,Metadata_image_set_name,Metadata_Time,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_plate,Metadata_Well,Metadata_number_of_singlecells,Metadata_compound,Metadata_dose,Metadata_control,...,Nuclei_Texture_Variance_CL_488_2_3_02_256,Nuclei_Texture_Variance_CL_488_2_3_03_256,Nuclei_Texture_Variance_CL_561_3_00_256,Nuclei_Texture_Variance_CL_561_3_01_256,Nuclei_Texture_Variance_CL_561_3_02_256,Nuclei_Texture_Variance_CL_561_3_03_256,Nuclei_Texture_Variance_DNA_3_00_256,Nuclei_Texture_Variance_DNA_3_01_256,Nuclei_Texture_Variance_DNA_3_02_256,Nuclei_Texture_Variance_DNA_3_03_256
3482,C-02_F0001,0001,289,43,1,C-02,162,Staurosporine,0.0,negative,...,0.000000,0.000000,0.002257,0.000000,0.021331,0.000000,0.000000,0.000000,0.00000,0.000000
3483,C-02_F0001,0001,135,84,1,C-02,162,Staurosporine,0.0,negative,...,0.821926,0.811037,0.000000,0.000000,0.000000,0.000000,0.738012,0.761422,0.73678,0.748586
3484,C-02_F0001,0001,1382,48,1,C-02,162,Staurosporine,0.0,negative,...,0.000000,0.000000,0.011763,0.018861,0.012985,0.004525,0.000000,0.000000,0.00000,0.000000
3485,C-02_F0001,0001,1206,65,1,C-02,162,Staurosporine,0.0,negative,...,0.187300,0.145600,0.463611,0.451690,0.482770,0.525575,0.000000,0.000000,0.00000,0.000000
3486,C-02_F0001,0001,1727,114,1,C-02,162,Staurosporine,0.0,negative,...,0.000000,0.000000,0.078314,0.061400,0.042139,0.049449,0.000000,0.000000,0.00000,0.000000


In [11]:
# save annotated df as parquet file
output(
    df=merged_df,
    output_filename=output_file,
    output_type="parquet",
)

'../data/annotated_data/run_20231017ChromaLive_6hr_4ch_MaxIP_sc.parquet'